In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.ml.feature_store import FeatureStore, Entity, FeatureView, CreationMode
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
fs = FeatureStore(
        session=session,
        database="INSURANCE",
        name="USER_ACTIVITY_FS",
        default_warehouse="COMPUTE_WH",
        creation_mode=CreationMode.CREATE_IF_NOT_EXIST,
     )

In [ ]:
user_entity = Entity(
    name="user_entity",
    join_keys=["user_id"]
)

fs.register_entity(user_entity)

In [ ]:
time_in_app_df = session.sql("""
    SELECT
        user_id,
        event_date,
        minutes_in_app,
        CASE WHEN minutes_in_app > 1 THEN 1 ELSE 0 END AS active_flag
    FROM insurance.operations.time_in_app
""")

time_in_app_fv = FeatureView(
    name="time_in_app_fv",
    entities=[user_entity],
    feature_df=time_in_app_df,
    timestamp_col="event_date",
    refresh_freq="1 DAY"
)

fs.register_feature_view(time_in_app_fv, version="3")

In [ ]:
messages_df = session.sql("""
    SELECT
        user_id,
        event_date,
        message_count
    FROM insurance.operations.messages
""")

messages_fv = FeatureView(
    name="messages_fv",
    entities=[user_entity],
    feature_df=messages_df,
    timestamp_col="event_date",
    refresh_freq="1 DAY"
)

fs.register_feature_view(messages_fv, version="3")

In [ ]:
calls_df = session.sql("""
    SELECT
        user_id,
        event_date,
        call_count
    FROM insurance.operations.calls
""")

calls_fv = FeatureView(
    name="calls_fv",
    entities=[user_entity],
    feature_df=calls_df,
    timestamp_col="event_date",
    refresh_freq="1 DAY"
)

fs.register_feature_view(calls_fv, version="3")

In [ ]:
spine_df = session.sql("""
    WITH base AS (
        SELECT DISTINCT user_id
        FROM insurance.operations.time_in_app
        WHERE event_date = '2024-11-16'
    )
    SELECT
        user_id,
        '2024-11-16'::DATE AS event_ts,
        (
            SELECT COUNT(*) 
            FROM insurance.operations.time_in_app t
            WHERE t.user_id = base.user_id
              AND t.event_date > '2024-11-16'
              AND t.event_date <= DATEADD(day, 7, '2024-11-16')
              AND t.minutes_in_app > 1
        ) AS active_next_7_days
    FROM base
""")
spine_df.show()

In [ ]:
time_in_app_fv=fs.get_feature_view(name="time_in_app_fv", version="3")
messages_fv=fs.get_feature_view(name="messages_fv", version="3")
calls_fv=fs.get_feature_view(name="calls_fv", version="3")

In [ ]:
training = fs.generate_training_set(
    spine_df=spine_df,
    spine_timestamp_col="event_ts",
    features=[time_in_app_fv, messages_fv, calls_fv],
    spine_label_cols=["active_next_7_days"],
    include_feature_view_timestamp_col=True
)
training.show()

In [ ]:
# df = training.to_pandas()

In [ ]:
timestamp_cols = [c for c in training.columns if c.endswith("_EVENT_DATE")]
print(timestamp_cols)

In [ ]:
for col in timestamp_cols:
    df = training.filter(
        training[col].is_null() | (training[col] == training["EVENT_TS"])
    )


In [ ]:
df.show()

In [ ]:
df = df.fillna(0)
df.head()